# Exercises XP Gold: W1_D4

## What You'll Learn

In this exercise, you will practice **inheritance** in Python.  
You will learn how to create base classes, extend them with new functionality, override methods, and combine multiple features through subclassing.  
You will also see how to integrate authentication into class methods and optionally build an interactive ATM system.

---

### Exercise 1: Bank Account

#### Part I — Basic BankAccount
- Create a class `BankAccount` with:
  - Attribute: `balance`
  - Method `__init__` to initialize the balance.
  - Method `deposit` to accept a positive integer and add it to the balance (raise an exception if not positive).
  - Method `withdraw` to accept a positive integer and deduct it from the balance (raise an exception if not positive).

#### Part II — MinimumBalanceAccount
- Create a subclass `MinimumBalanceAccount` that inherits from `BankAccount`.
- Extend `__init__` to accept `minimum_balance` (default 0).
- Override `withdraw` to allow withdrawals only if the balance remains above `minimum_balance`.

#### Part III — Authentication
- Add attributes to `BankAccount`:
  - `username`
  - `password`
  - `authenticated` (default False)
- Add a method `authenticate(username, password)` to set `authenticated` to True if credentials match.
- Modify `deposit` and `withdraw` to only work if `authenticated` is True.

#### Part IV (BONUS) — ATM
- Create an `ATM` class that:
  - Accepts a list of account objects and a `try_limit` in `__init__`.
  - Displays a main menu (`show_main_menu`) to log in or exit.
  - Allows login attempts until `try_limit` is reached (`log_in`).
  - Once logged in, shows an account menu (`show_account_menu`) to deposit, withdraw, or exit.

## Part I — BankAccount (balance, deposit, withdraw)

In [1]:
# This class models a basic bank account with a balance and
# methods to deposit and withdraw positive amounts.

class BankAccount:
    def __init__(self, balance: int = 0):
        """Initialize the account with a starting balance (integer)."""
        if not isinstance(balance, int):
            raise TypeError("Balance must be an integer.")
        if balance < 0:
            raise ValueError("Balance cannot be negative.")
        self.balance = balance

    def deposit(self, amount: int) -> None:
        """Deposit a positive integer amount into the account."""
        if not isinstance(amount, int):
            raise TypeError("Deposit amount must be an integer.")
        if amount <= 0:
            raise ValueError("Deposit amount must be positive.")
        self.balance += amount

    def withdraw(self, amount: int) -> None:
        """Withdraw a positive integer amount from the account."""
        if not isinstance(amount, int):
            raise TypeError("Withdrawal amount must be an integer.")
        if amount <= 0:
            raise ValueError("Withdrawal amount must be positive.")
        if amount > self.balance:
            raise ValueError("Insufficient funds.")
        self.balance -= amount

## Part II — MinimumBalanceAccount (inheritance + override withdraw)

In [2]:
# This account enforces a minimum balance that must remain after withdrawal.

class MinimumBalanceAccount(BankAccount):
    def __init__(self, balance: int = 0, minimum_balance: int = 0):
        """Initialize with a starting balance and a minimum required balance."""
        super().__init__(balance=balance)
        if not isinstance(minimum_balance, int):
            raise TypeError("minimum_balance must be an integer.")
        if minimum_balance < 0:
            raise ValueError("minimum_balance cannot be negative.")
        self.minimum_balance = minimum_balance

    def withdraw(self, amount: int) -> None:
        """Allow withdraw only if balance - amount >= minimum_balance."""
        if not isinstance(amount, int):
            raise TypeError("Withdrawal amount must be an integer.")
        if amount <= 0:
            raise ValueError("Withdrawal amount must be positive.")
        if self.balance - amount < self.minimum_balance:
            raise ValueError("Withdrawal would violate the minimum balance.")
        self.balance -= amount

## Part III — Add authentication to BankAccount

In [3]:
# We extend BankAccount to support username/password and an authenticated flag.
# For simplicity, we implement this as a mix-in style subclass.

class AuthenticatedBankAccount(BankAccount):
    def __init__(self, username: str, password: str, balance: int = 0):
        """Initialize with credentials and starting balance; authenticated defaults to False."""
        super().__init__(balance=balance)
        if not isinstance(username, str) or not isinstance(password, str):
            raise TypeError("username and password must be strings.")
        if not username or not password:
            raise ValueError("username and password cannot be empty.")
        self.username = username
        self.password = password
        self.authenticated = False

    def authenticate(self, username: str, password: str) -> bool:
        """Set authenticated=True if credentials match; return True/False."""
        if username == self.username and password == self.password:
            self.authenticated = True
            return True
        self.authenticated = False
        return False

    # Override deposit/withdraw to require authentication first
    def deposit(self, amount: int) -> None:
        """Deposit only if authenticated; otherwise raise an Exception."""
        if not self.authenticated:
            raise PermissionError("Not authenticated.")
        super().deposit(amount)

    def withdraw(self, amount: int) -> None:
        """Withdraw only if authenticated; otherwise raise an Exception."""
        if not self.authenticated:
            raise PermissionError("Not authenticated.")
        super().withdraw(amount)


class AuthenticatedMinimumBalanceAccount(AuthenticatedBankAccount, MinimumBalanceAccount):
    """
    Multiple inheritance:
    - Authenticated behavior (credentials + checks before actions)
    - Minimum balance rules for withdraw
    MRO ensures MinimumBalanceAccount.withdraw logic applies when authenticated.
    """
    def __init__(self, username: str, password: str, balance: int = 0, minimum_balance: int = 0):
        # Initialize both sides properly via explicit calls then fix MRO state:
        MinimumBalanceAccount.__init__(self, balance=balance, minimum_balance=minimum_balance)
        # After setting balance/minimum_balance, initialize auth part
        AuthenticatedBankAccount.__init__(self, username=username, password=password, balance=self.balance)

    # Ensure deposit/withdraw still require auth; withdraw uses MinimumBalanceAccount's rule.
    def withdraw(self, amount: int) -> None:
        if not self.authenticated:
            raise PermissionError("Not authenticated.")
        MinimumBalanceAccount.withdraw(self, amount)

## Quick tests for Parts I-III

In [4]:
# Part I
acc = BankAccount(100)
acc.deposit(50)
acc.withdraw(30)
print("Part I balance:", acc.balance)  # 120

# Part II
mba = MinimumBalanceAccount(balance=200, minimum_balance=50)
mba.withdraw(100)
print("Part II balance:", mba.balance)  # 100
# mba.withdraw(60)  # Uncomment to see error (would drop below min balance)

# Part III
aacc = AuthenticatedBankAccount(username="alice", password="pw123", balance=100)
try:
    aacc.deposit(10)  # should raise (not authenticated)
except PermissionError as e:
    print("Expected:", e)

ok = aacc.authenticate("alice", "pw123")
print("Auth success:", ok)
aacc.deposit(10)
aacc.withdraw(20)
print("Part III balance:", aacc.balance)  # 90

# Part III + Min Balance
am = AuthenticatedMinimumBalanceAccount(username="bob", password="pw", balance=300, minimum_balance=100)
print("Auth before:", am.authenticated)
am.authenticate("bob", "pw")
print("Auth after:", am.authenticated)
am.withdraw(150)  # leaves 150 >= 100
print("Part III+II balance:", am.balance)  # 150

Part I balance: 120
Part II balance: 100
Expected: Not authenticated.
Auth success: True
Part III balance: 90
Auth before: False
Auth after: True
Part III+II balance: 150


## Part IV (Bonus) — ATM class with login and account menu

In [5]:
from typing import List, Union

AccountType = Union[AuthenticatedBankAccount, AuthenticatedMinimumBalanceAccount]

class ATM:
    def __init__(self, account_list: List[AccountType], try_limit: int = 2):
        """Validate inputs, set state, then show the main menu."""
        # Validate account_list contains proper instances
        if not isinstance(account_list, list) or not all(
            isinstance(a, (AuthenticatedBankAccount, AuthenticatedMinimumBalanceAccount)) for a in account_list
        ):
            raise TypeError("account_list must be a list of authenticated account instances.")

        self.accounts = account_list

        # Validate try_limit: if invalid, raise then fallback to 2
        try:
            if not isinstance(try_limit, int) or try_limit <= 0:
                raise ValueError("try_limit must be a positive integer.")
            self.try_limit = try_limit
        except Exception as e:
            print(f"Invalid try_limit ({e}). Fallback to 2.")
            self.try_limit = 2

        self.current_tries = 0
        self.show_main_menu()

    def show_main_menu(self):
        """Display main menu in a loop: Log in or Exit."""
        while True:
            print("\n=== ATM Main Menu ===")
            print("1) Log in")
            print("2) Exit")
            choice = input("Select an option (1/2): ").strip()
            if choice == "1":
                username = input("Username: ").strip()
                password = input("Password: ").strip()
                self.log_in(username, password)
            elif choice == "2":
                print("Goodbye!")
                break
            else:
                print("Invalid selection. Try again.")

    def log_in(self, username: str, password: str):
        """
        Try to authenticate against all accounts. If match, show account menu.
        Otherwise, increment tries until reaching try_limit; then exit.
        """
        # Try each account
        for account in self.accounts:
            if account.authenticate(username, password):
                print(f"Login successful. Welcome, {username}.")
                self.current_tries = 0  # reset tries
                return self.show_account_menu(account)

        # If we reach here, no match
        self.current_tries += 1
        print(f"Login failed. Attempt {self.current_tries}/{self.try_limit}.")
        if self.current_tries >= self.try_limit:
            print("Max tries reached. Shutting down.")
            raise SystemExit

    def show_account_menu(self, account: AccountType):
        """Loop to let the user deposit, withdraw, or exit for the logged-in account."""
        while True:
            print("\n--- Account Menu ---")
            print(f"Current balance: {account.balance}")
            print("1) Deposit")
            print("2) Withdraw")
            print("3) Exit to main menu")
            choice = input("Select an option (1/2/3): ").strip()

            if choice == "1":
                try:
                    amt = int(input("Amount to deposit (int): ").strip())
                    account.deposit(amt)
                    print("Deposit successful.")
                except Exception as e:
                    print("Error:", e)

            elif choice == "2":
                try:
                    amt = int(input("Amount to withdraw (int): ").strip())
                    account.withdraw(amt)
                    print("Withdrawal successful.")
                except Exception as e:
                    print("Error:", e)

            elif choice == "3":
                print("Returning to main menu.")
                break
            else:
                print("Invalid selection. Try again.")

In [8]:
# Create test accounts
acc1 = AuthenticatedBankAccount(username="alice", password="pw123", balance=500)
acc2 = AuthenticatedMinimumBalanceAccount(username="bob", password="pw", balance=300, minimum_balance=100)

# Start the ATM with these accounts and a try limit of 2
# atm = ATM([acc1, acc2], try_limit=2)

## Conclusion

In this exercise, I practiced:

- Creating a **base class** with attributes and methods.
- Using **inheritance** to create specialized subclasses.
- **Overriding methods** to modify or extend functionality in subclasses.
- Adding **authentication** to control access to sensitive operations like deposit and withdraw.
- Using **multiple inheritance** to combine different behaviors (minimum balance + authentication).
- (Bonus) Designing an interactive **ATM** system to integrate these features in a practical application.

This project helped me understand how inheritance can simplify code by reusing and extending base functionality, and how to combine multiple class features into a single cohesive object.